In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [26]:
def obtener_fecha(df):
    df['fecha'] = pd.to_datetime(df['validTimeLocal'], utc=True)
    df = df.sort_values(by=['fecha'])
    return df

In [27]:
def descomponer_fecha(df):
    df['hora'] = df['fecha'].dt.hour
    df['dia'] = df['fecha'].dt.day
    df['mes'] = df['fecha'].dt.month
    df['anio'] = df['fecha'].dt.year
    return df

In [28]:
def obtener_columnas_relevantes(df):
    df = df.rename(columns={'temperature': 'temp', 
                            'temperatureDewPoint': 'dewPt', 
                            'temperatureHeatIndex':'heat_index', 
                            'relativeHumidity':'rh', 
                            'pressureMeanSeaLevel':'pressure', 
                            'visibility':'vis', 
                            'temperatureWindChill':'wc', 
                            'temperatureFeelsLike':'feels_like',
                            'uvIndex':'uv_index',
                            'validTimeUtc': 'valid_time_gmt'})
    df = df[['hora', 'dia', 'mes', 'anio', 'temp', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 'feels_like', 'uv_index', 'valid_time_gmt', 'windDirectionCardinal']]
    return df

In [29]:
def preprocess(data, fecha_pred):
    data = obtener_fecha(data)
    data['antelacion'] = (pd.to_datetime(data['fecha']) - fecha_pred).dt.total_seconds() // 3600
    data = descomponer_fecha(data)
    data = obtener_columnas_relevantes(data)
    return data

In [30]:
data = pd.read_json(r"../secundaria-raw/13h_03_05_2024.json")
data

,cloudCover,dayOfWeek,dayOrNight,expirationTimeUtc,iconCode,iconCodeExtend,precipChance,precipType,pressureMeanSeaLevel,qpf,...,validTimeLocal,validTimeUtc,visibility,windDirection,windDirectionCardinal,windGust,windSpeed,wxPhraseLong,wxPhraseShort,wxSeverity
0,78,Friday,D,1714734519,28,2800,15,rain,29.79,0.00,...,2024-05-03T13:00:00+0100,1714737600,10.0,263,W,NaN,6,Mostly Cloudy,M Cloudy,1
1,81,Friday,D,1714734519,26,2600,15,rain,29.80,0.00,...,2024-05-03T14:00:00+0100,1714741200,10.0,258,WSW,NaN,6,Cloudy,Cloudy,1
2,98,Friday,D,1714734519,11,1100,46,rain,29.80,0.01,...,2024-05-03T15:00:00+0100,1714744800,7.0,255,WSW,NaN,8,Showers,Showers,1
3,98,Friday,D,1714734519,26,2600,24,rain,29.80,0.00,...,2024-05-03T16:00:00+0100,1714748400,6.0,249,WSW,NaN,9,Cloudy,Cloudy,1
4,98,Friday,D,1714734519,11,4600,33,rain,29.80,0.00,...,2024-05-03T17:00:00+0100,1714752000,6.0,243,WSW,NaN,8,Few Showers,Few Showers,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,67,Saturday,D,1714735978,11,1100,43,rain,29.85,0.00,...,2024-05-18T08:00:00+0100,1716015600,7.0,282,WNW,NaN,11,Showers,Showers,1
356,68,Saturday,D,1714735978,11,1100,41,rain,29.85,0.00,...,2024-05-18T09:00:00+0100,1716019200,7.0,279,W,NaN,11,Showers,Showers,1
357,70,Saturday,D,1714735978,11,1100,38,rain,29.86,0.00,...,2024-05-18T10:00:00+0100,1716022800,7.0,276,W,NaN,11,Showers,Showers,1
358,71,Saturday,D,1714735978,11,1100,36,rain,29.87,0.00,...,2024-05-18T11:00:00+0100,1716026400,7.0,263,W,NaN,12,Showers,Showers,1


In [31]:
dirName = '../secundaria-raw'
directory = os.fsencode(dirName)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    data = preprocess(data, datetime.strptime(filename.replace('h', ''), '%H_%d_%m_%Y.json'))
    data

TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects